In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
data=pd.read_csv("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
data.head(3)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."


In [3]:
sentiment_data = data[['reviews.rating', 'reviews.text', 'reviews.title']]
sentiment_data.head()

,reviews.rating,reviews.text,reviews.title
0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...
1,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...
2,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...
3,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...
4,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...


In [4]:
## prepare data
def sentiment_labeler(score):
    if (score==5) or (score==4):
        return "Positive"
    elif (score==3):
        return "Neutral"
    else:
        return "Negative"
    
sentiment_data["sentiment_label"]=sentiment_data["reviews.rating"].apply(sentiment_labeler)
sentiment_data.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,reviews.rating,reviews.text,reviews.title,sentiment_label
0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Neutral
1,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,Positive
2,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,Positive
3,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Positive
4,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Positive


### prepare train, test data


In [5]:
## set up stanfordcorenlp

In [6]:
from pycorenlp import StanfordCoreNLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import numpy as np
import string

In [7]:
## helper function to extract pos tag features

def extract_POS(statements):
    print('Extracting POS Tags')
    pos_tags = POS_tagging(statements,return_word_tag_pairs=False)
    bigrams_pos = POS_groupping(pos_tags, grams=2)
    trigrams_pos =POS_groupping(pos_tags, grams=3)
    print('Finished')
    return pos_tags,bigrams_pos,trigrams_pos

##
def POS_tagging(statements, return_word_tag_pairs = False):
    core_nlp = StanfordCoreNLP('http://localhost:9000')
    print("NLP_Task ready to use.")
    POS_tags = list()
    for statement in statements:
        statement_tags = list()
        annotations = core_nlp.annotate(statement, properties={
            'annotators': 'tokenize,pos',
            'outputFormat': 'json'
            })
        for output in annotations['sentences']:
            statement_tags.append('<s>')
            previous = ''
            for token in output['tokens']:
                if return_word_tag_pairs:
                    statement_tags.append(token['word']+'/'+token['pos'])
                else:
                    statement_tags.append(token['pos'])

        POS_tags.append(statement_tags)
    return POS_tags

## 
def POS_groupping(sentences_pos,grams=1):
    result = list()
    for sentence_tags in sentences_pos:
        tag_group = list()
        for index, each_tag in enumerate(sentence_tags):
            if index < len(sentence_tags)-grams and len(sentence_tags)>=grams:
                format_str = str()
                for i in range(0,grams):
                    format_str += sentence_tags[index+i]
                    if i<grams-1:
                        format_str += '_'
                tag_group.append(format_str)
        result.append(tag_group)
    return result

##
def RemoveConsecutiveTags(list_to_remove, postags,ignore_punctuation=False):
    withoutConsecutiveTags = list()
    for each_tag in postags:
        removed = list()
        previous = ''
        for tt in each_tag:
            if tt != previous:
                if not ignore_punctuation:
                    removed.append(tt)
                elif tt not in string.punctuation:
                    removed.append(tt)
                previous = tt
            elif tt not in list_to_remove:
                removed.append(tt)
                previous = tt
        withoutConsecutiveTags.append(removed)
    return withoutConsecutiveTags

In [8]:
## extracting pos-tag features
unigram_pos_tr, bigrams_pos_tr, trigram_pos_tr = extract_POS(statements=sentiment_data['reviews.text'])
unigram_pos_ts, bigrams_pos_ts, trigram_pos_ts = extract_POS(statements=sentiment_data['reviews.text'])

Extracting POS Tags
NLP_Task ready to use.
Finished
Extracting POS Tags
NLP_Task ready to use.
Finished


In [9]:
## remove duplicated pos tag

# For review data
list_to_remove = ['NNP','CD']

removed_pos_tr =RemoveConsecutiveTags(list_to_remove,unigram_pos_tr)
removed_pos_bigrams_tr = POS_groupping(grams=2,sentences_pos=removed_pos_tr)
removed_pos_trigrams_tr = POS_groupping(grams=3,sentences_pos=removed_pos_tr)

removed_pos_ts =RemoveConsecutiveTags(list_to_remove,unigram_pos_ts)
removed_pos_bigrams_ts = POS_groupping(grams=2,sentences_pos=removed_pos_ts)
removed_pos_trigrams_ts = POS_groupping(grams=3,sentences_pos=removed_pos_ts)

In [10]:
sentiment_feats= pd.DataFrame()
sentiment_feats['reviews'] = sentiment_data['reviews.text']
sentiment_feats['label'] = sentiment_data['sentiment_label']
sentiment_feats['reviews.rating'] = sentiment_data['reviews.rating']

list_to_remove = ['NNP','CD']

sentiment_feats['pos_unig_tr'] = pd.Series([" ".join(x).replace('<s>','').replace('$','dollar').strip() for x in removed_pos_tr])
sentiment_feats['pos_big_tr'] = pd.Series([" ".join(x).replace('$','dollar').strip() for x in removed_pos_bigrams_tr])
sentiment_feats['pos_trig_tr'] = pd.Series([" ".join(x).replace('$','dollar').strip() for x in removed_pos_trigrams_tr])
sentiment_feats.head()

,reviews,label,reviews.rating,pos_unig_tr,pos_big_tr,pos_trig_tr
0,I order 3 of them and one of the item is bad q...,Neutral,3,PRP VBP CD IN PRP CC CD IN DT NN VBZ JJ NN . ...,<s>_PRP PRP_VBP VBP_CD CD_IN IN_PRP PRP_CC CC_...,<s>_PRP_VBP PRP_VBP_CD VBP_CD_IN CD_IN_PRP IN_...
1,Bulk is always the less expensive way to go fo...,Positive,4,NN VBZ RB DT RBR JJ NN TO VB IN NNS IN DT,<s>_NN NN_VBZ VBZ_RB RB_DT DT_RBR RBR_JJ JJ_NN...,<s>_NN_VBZ NN_VBZ_RB VBZ_RB_DT RB_DT_RBR DT_RB...
2,Well they are not Duracell but for the price i...,Positive,5,RB PRP VBP RB NNP CC IN DT NN FW VBP JJ .,<s>_RB RB_PRP PRP_VBP VBP_RB RB_NNP NNP_CC CC_...,<s>_RB_PRP RB_PRP_VBP PRP_VBP_RB VBP_RB_NNP RB...
3,Seem to work as well as name brand batteries a...,Positive,5,NNP TO VB RB RB IN NN NN NNS IN DT RB JJR NN,<s>_NNP NNP_TO TO_VB VB_RB RB_RB RB_IN IN_NN N...,<s>_NNP_TO NNP_TO_VB TO_VB_RB VB_RB_RB RB_RB_I...
4,These batteries are very long lasting the pric...,Positive,5,DT NNS VBP RB JJ VBG DT NN VBZ JJ .,<s>_DT DT_NNS NNS_VBP VBP_RB RB_JJ JJ_VBG VBG_...,<s>_DT_NNS DT_NNS_VBP NNS_VBP_RB VBP_RB_JJ RB_...


In [11]:
###
def GetFeaturesFromPOS(training_data, user_defined_vocabulary=None):
    user_defined_vocabulary = [x.lower().replace('$','dollar') for x in user_defined_vocabulary]

    # making string of the data
    training_str = [" ".join(x) for x in training_data]

    #replace $ by dollar
    training_str = [x.replace('$', 'dollar').replace('<s>','sos') for x in training_str]

    # features using binary iformation
    oneHotVectorizer = CountVectorizer(vocabulary=user_defined_vocabulary,binary=True)
    tr_onehot = oneHotVectorizer.fit_transform(training_str).toarray()
    print(oneHotVectorizer.vocabulary_)

    # features using no-binary information (counting)
    countVectorizer = CountVectorizer(vocabulary=user_defined_vocabulary,binary=True)
    tr_count = countVectorizer.fit_transform(training_str).toarray()

    # features using tf-idf vectors
    tfIdfVectorizer = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)
    tr_tfidf = tfIdfVectorizer.fit_transform(tr_count)

    return tr_onehot, tr_count, tr_tfidf

In [12]:
pos_relevant_unigrams =  ['VBZ', 'DT', 'NNPS', 'VBP', 'JJ', 'IN', 'WRB', 'VBD', 'PRP', 'RP', 'WDT', 'VB', 'NNP', 'VBG', 'PRP$', 'VBN', 'CD', 'RB', 'WP', 'JJS', 'JJR', 'EX', 'RBS', 'FW', 'LS'] 
amazonRev_onehot_unigram_tr, amazonRev_count_unigram_tr, amazonRev_tfidf_unigram_tr = GetFeaturesFromPOS(training_data=removed_pos_tr, user_defined_vocabulary=pos_relevant_unigrams)

pos_relevant_bigrams = ['NNPS_VBP', 'VB_NNP', 'IN_DT', 'VB_JJ', 'JJ_CD', 'CD_NNS', 'DT_JJS', 'JJR_IN', 'IN_CD', 'CC_IN', 'RB_VBD', 'CD_NN', 'NN_TO', 'JJR_JJ', 'VB_CD'] 
amazonRev_onehot_bigram_tr, amazonRev_count_bigram_tr, amazonRev_tfidf_bigram_tr = GetFeaturesFromPOS(training_data=removed_pos_bigrams_tr, user_defined_vocabulary=pos_relevant_bigrams)

pos_relevant_trigrams = ['VBD_VBN_IN', 'IN_DT_JJ', 'CD_NN_IN', 'IN_CD_NNS', 'IN_DT_NN', 'DT_JJ_CD', 'MD_VB_IN', 'JJS_JJ_NN', 'CC_JJ_NNS', 'JJ_NNS_VBP', 'VBP_CD_NN', 'sos_JJR_IN', 'IN_DT_NNS','JJ_NN_MD']
amazonRev_onehot_trigram_tr, amazonRev_count_trigram_tr, amazonRev_tfidf_trigram_tr = GetFeaturesFromPOS(training_data=removed_pos_trigrams_tr, user_defined_vocabulary=pos_relevant_trigrams)

{'vbz': 0, 'dt': 1, 'nnps': 2, 'vbp': 3, 'jj': 4, 'in': 5, 'wrb': 6, 'vbd': 7, 'prp': 8, 'rp': 9, 'wdt': 10, 'vb': 11, 'nnp': 12, 'vbg': 13, 'prpdollar': 14, 'vbn': 15, 'cd': 16, 'rb': 17, 'wp': 18, 'jjs': 19, 'jjr': 20, 'ex': 21, 'rbs': 22, 'fw': 23, 'ls': 24}
{'nnps_vbp': 0, 'vb_nnp': 1, 'in_dt': 2, 'vb_jj': 3, 'jj_cd': 4, 'cd_nns': 5, 'dt_jjs': 6, 'jjr_in': 7, 'in_cd': 8, 'cc_in': 9, 'rb_vbd': 10, 'cd_nn': 11, 'nn_to': 12, 'jjr_jj': 13, 'vb_cd': 14}
{'vbd_vbn_in': 0, 'in_dt_jj': 1, 'cd_nn_in': 2, 'in_cd_nns': 3, 'in_dt_nn': 4, 'dt_jj_cd': 5, 'md_vb_in': 6, 'jjs_jj_nn': 7, 'cc_jj_nns': 8, 'jj_nns_vbp': 9, 'vbp_cd_nn': 10, 'sos_jjr_in': 11, 'in_dt_nns': 12, 'jj_nn_md': 13}


In [13]:

sentiment_feats['pos_unigrams_1hot_tr'] =  pd.Series([np.array(x) for x in amazonRev_onehot_unigram_tr])
sentiment_feats['pos_bigrams_1hot_tr'] = pd.Series([np.array(x) for x in amazonRev_onehot_bigram_tr])
sentiment_feats['pos_trigrams_1hot_tr'] = pd.Series([np.array(x) for x in amazonRev_onehot_trigram_tr])

sentiment_feats['pos_unigrams_count_tr'] = pd.Series([np.array(x) for x in amazonRev_count_unigram_tr])
sentiment_feats['pos_bigrams_count_tr'] = pd.Series([np.array(x) for x in amazonRev_count_bigram_tr])
sentiment_feats['pos_trigrams_count_tr'] = pd.Series([np.array(x) for x in amazonRev_count_trigram_tr])

sentiment_feats['pos_unigrams_tfidf_tr'] =  pd.Series([np.array(x) for x in amazonRev_tfidf_unigram_tr])
sentiment_feats['pos_bigrams_tfidf_tr'] = pd.Series([np.array(x) for x in amazonRev_tfidf_bigram_tr])
sentiment_feats['pos_trigrams_tfidf_tr'] = pd.Series([np.array(x) for x in amazonRev_tfidf_trigram_tr])

sentiment_feats.head()

,reviews,label,reviews.rating,pos_unig_tr,pos_big_tr,pos_trig_tr,pos_unigrams_1hot_tr,pos_bigrams_1hot_tr,pos_trigrams_1hot_tr,pos_unigrams_count_tr,pos_bigrams_count_tr,pos_trigrams_count_tr,pos_unigrams_tfidf_tr,pos_bigrams_tfidf_tr,pos_trigrams_tfidf_tr
0,I order 3 of them and one of the item is bad q...,Neutral,3,PRP VBP CD IN PRP CC CD IN DT NN VBZ JJ NN . ...,<s>_PRP PRP_VBP VBP_CD CD_IN IN_PRP PRP_CC CC_...,<s>_PRP_VBP PRP_VBP_CD VBP_CD_IN CD_IN_PRP IN_...,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 16)\t0.468652052876125\n (0, 13)\t0.457...","(0, 12)\t0.8773340588158026\n (0, 2)\t0.479...","(0, 4)\t1.0"
1,Bulk is always the less expensive way to go fo...,Positive,4,NN VBZ RB DT RBR JJ NN TO VB IN NNS IN DT,<s>_NN NN_VBZ VBZ_RB RB_DT DT_RBR RBR_JJ JJ_NN...,<s>_NN_VBZ NN_VBZ_RB VBZ_RB_DT RB_DT_RBR DT_RB...,"[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 17)\t0.4098436732684944\n (0, 11)\t0.45...","(0, 12)\t1.0",
2,Well they are not Duracell but for the price i...,Positive,5,RB PRP VBP RB NNP CC IN DT NN FW VBP JJ .,<s>_RB RB_PRP PRP_VBP VBP_RB RB_NNP NNP_CC CC_...,<s>_RB_PRP RB_PRP_VBP PRP_VBP_RB VBP_RB_NNP RB...,"[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 23)\t0.7435935756396004\n (0, 17)\t0.23...","(0, 9)\t0.9124980165040811\n (0, 2)\t0.4090...","(0, 4)\t1.0"
3,Seem to work as well as name brand batteries a...,Positive,5,NNP TO VB RB RB IN NN NN NNS IN DT RB JJR NN,<s>_NNP NNP_TO TO_VB VB_RB RB_RB RB_IN IN_NN N...,<s>_NNP_TO NNP_TO_VB TO_VB_RB VB_RB_RB RB_RB_I...,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 20)\t0.6457240570351381\n (0, 17)\t0.31...","(0, 2)\t1.0",
4,These batteries are very long lasting the pric...,Positive,5,DT NNS VBP RB JJ VBG DT NN VBZ JJ .,<s>_DT DT_NNS NNS_VBP VBP_RB RB_JJ JJ_VBG VBG_...,<s>_DT_NNS DT_NNS_VBP NNS_VBP_RB VBP_RB_JJ RB_...,"[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 17)\t0.34941682605529095\n (0, 13)\t0.5...",,


In [14]:
## Reshape the tfidf vectors so they can be combined
sentiment_feats['pos_unigrams_tfidf'] = sentiment_feats['pos_unigrams_tfidf_tr'].apply(lambda x: x.reshape((1,))[0].toarray()[0])
sentiment_feats['pos_bigrams_tfidf'] = sentiment_feats['pos_bigrams_tfidf_tr'].apply(lambda x: x.reshape((1,))[0].toarray()[0])
sentiment_feats['pos_trigrams_tfidf'] = sentiment_feats['pos_trigrams_tfidf_tr'].apply(lambda x: x.reshape((1,))[0].toarray()[0])

sentiment_feats.head()

,reviews,label,reviews.rating,pos_unig_tr,pos_big_tr,pos_trig_tr,pos_unigrams_1hot_tr,pos_bigrams_1hot_tr,pos_trigrams_1hot_tr,pos_unigrams_count_tr,pos_bigrams_count_tr,pos_trigrams_count_tr,pos_unigrams_tfidf_tr,pos_bigrams_tfidf_tr,pos_trigrams_tfidf_tr,pos_unigrams_tfidf,pos_bigrams_tfidf,pos_trigrams_tfidf
0,I order 3 of them and one of the item is bad q...,Neutral,3,PRP VBP CD IN PRP CC CD IN DT NN VBZ JJ NN . ...,<s>_PRP PRP_VBP VBP_CD CD_IN IN_PRP PRP_CC CC_...,<s>_PRP_VBP PRP_VBP_CD VBP_CD_IN CD_IN_PRP IN_...,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 16)\t0.468652052876125\n (0, 13)\t0.457...","(0, 12)\t0.8773340588158026\n (0, 2)\t0.479...","(0, 4)\t1.0","[0.33326393978382374, 0.25145369000993206, 0.0...","[0.0, 0.0, 0.4798801404953009, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Bulk is always the less expensive way to go fo...,Positive,4,NN VBZ RB DT RBR JJ NN TO VB IN NNS IN DT,<s>_NN NN_VBZ VBZ_RB RB_DT DT_RBR RBR_JJ JJ_NN...,<s>_NN_VBZ NN_VBZ_RB VBZ_RB_DT RB_DT_RBR DT_RB...,"[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 17)\t0.4098436732684944\n (0, 11)\t0.45...","(0, 12)\t1.0",,"[0.49362715060265006, 0.37245064254068355, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Well they are not Duracell but for the price i...,Positive,5,RB PRP VBP RB NNP CC IN DT NN FW VBP JJ .,<s>_RB RB_PRP PRP_VBP VBP_RB RB_NNP NNP_CC CC_...,<s>_RB_PRP RB_PRP_VBP PRP_VBP_RB VBP_RB_NNP RB...,"[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 23)\t0.7435935756396004\n (0, 17)\t0.23...","(0, 9)\t0.9124980165040811\n (0, 2)\t0.4090...","(0, 4)\t1.0","[0.0, 0.21698552304929278, 0.0, 0.294983793842...","[0.0, 0.0, 0.4090811287215748, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Seem to work as well as name brand batteries a...,Positive,5,NNP TO VB RB RB IN NN NN NNS IN DT RB JJR NN,<s>_NNP NNP_TO TO_VB VB_RB RB_RB RB_IN IN_NN N...,<s>_NNP_TO NNP_TO_VB TO_VB_RB VB_RB_RB RB_RB_I...,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 20)\t0.6457240570351381\n (0, 17)\t0.31...","(0, 2)\t1.0",,"[0.0, 0.2847429487413753, 0.0, 0.0, 0.0, 0.276...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,These batteries are very long lasting the pric...,Positive,5,DT NNS VBP RB JJ VBG DT NN VBZ JJ .,<s>_DT DT_NNS NNS_VBP VBP_RB RB_JJ JJ_VBG VBG_...,<s>_DT_NNS DT_NNS_VBP NNS_VBP_RB VBP_RB_JJ RB_...,"[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(0, 17)\t0.34941682605529095\n (0, 13)\t0.5...",,,"[0.4208473705175385, 0.3175369777968061, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [16]:
# Chunker class to build parse trees

import nltk

class Chunker():
	def __init__(self, n):
		# Load the corpus to train the chunk tagger
		conll = nltk.corpus.conll2000.chunked_sents()
		treebank = nltk.corpus.treebank_chunk.chunked_sents()
		data = conll + treebank

		chunks = [ nltk.chunk.tree2conlltags(tree) for tree in data ]
		chunks = [ [(tag[1], tag[2]) for tag in tags] for tags in chunks ]

		train_chunks = chunks

		# Train the chunk tagger
		self.chunk_tagger = None
		if n == "Unigram":
			self.chunk_tagger = nltk.tag.UnigramTagger(train_chunks)
		elif n == "Bigram":
			self.chunk_tagger = nltk.tag.BigramTagger(train_chunks)
		elif n == "Trigram":
			self.chunk_tagger = nltk.tag.TrigramTagger(train_chunks)
		else:
			chunker = nltk.tag.UnigramTagger(train_chunks)
			chunker = nltk.tag.BigramTagger(train_chunks, backoff=chunker)
			chunker = nltk.tag.TrigramTagger(train_chunks, backoff=chunker)
			self.chunk_tagger = chunker

	def parseTree(self, tokens):
		# Gather the words, tags, and chunks
		words = [w for (w,t) in tokens]
		tags = [t for (w,t) in tokens]
		chunks = self.chunk_tagger.tag(tags)

		# Sanity check
		assert len(words) == len(tags)
		assert len(words) == len(chunks)

		# Build the parse tree
		l = []
		for i in range(0,len(words)):
			if chunks[i][1]:
				l.append( ' '.join([words[i], tags[i], chunks[i][1]]) )

		return nltk.chunk.conllstr2tree('\n'.join(l))

def chunkLabels(tree):
	l = []
	for t in tree.subtrees():
		if t.label() != 'S':
			l.append(t.label())
	return ' '.join(l)

In [28]:
## Parse into chunks and extract chunk labels

ch_unigram = Chunker('unigram')
ch_bigram = Chunker('bigram')
ch_trigram = Chunker('trigram')

sentiment_feats['uni_chunk_labels'] = sentiment_feats['reviews'].apply(str.split)
sentiment_feats['uni_chunk_labels'] = sentiment_feats['uni_chunk_labels'].apply(nltk.pos_tag)
sentiment_feats['uni_chunk_labels'] = sentiment_feats['uni_chunk_labels'].apply(ch_unigram.parseTree)
sentiment_feats['uni_chunk_labels'] = sentiment_feats['uni_chunk_labels'].apply(chunkLabels)

sentiment_feats['bi_chunk_labels'] = sentiment_feats['reviews'].apply(str.split)
sentiment_feats['bi_chunk_labels'] = sentiment_feats['bi_chunk_labels'].apply(nltk.pos_tag)
sentiment_feats['bi_chunk_labels'] = sentiment_feats['bi_chunk_labels'].apply(ch_bigram.parseTree)
sentiment_feats['bi_chunk_labels'] = sentiment_feats['bi_chunk_labels'].apply(chunkLabels)

sentiment_feats['tri_chunk_labels'] = sentiment_feats['reviews'].apply(str.split)
sentiment_feats['tri_chunk_labels'] = sentiment_feats['tri_chunk_labels'].apply(nltk.pos_tag)
sentiment_feats['tri_chunk_labels'] = sentiment_feats['tri_chunk_labels'].apply(ch_trigram.parseTree)
sentiment_feats['tri_chunk_labels'] = sentiment_feats['tri_chunk_labels'].apply(chunkLabels)

sentiment_feats[['uni_chunk_labels', 'bi_chunk_labels', 'tri_chunk_labels']].head()

,uni_chunk_labels,bi_chunk_labels,tri_chunk_labels
0,NP PP NP NP NP VP NP VP NP NP VP NP PP NP PP N...,NP PP NP NP NP VP NP VP NP NP VP NP PP NP PP N...,NP PP NP NP NP VP NP VP NP NP VP NP PP NP PP N...
1,NP VP NP NP PP NP PP NP PP NP,NP VP NP NP PP NP PP NP PP NP,NP VP NP NP PP NP PP NP PP NP
2,NP VP NP NP VP NP,NP VP NP NP VP NP,NP VP NP NP VP NP
3,NP VP NP PP NP NP,NP VP NP PP NP NP,NP VP NP PP NP NP
4,NP VP VP NP VP,NP VP VP NP VP,NP VP VP NP VP


In [21]:
def countChunks(data):
    tokens = data.split()
    vocabulary = ['NP', 'VP', 'PP']
    counts = [0, 0, 0, 0]
    for t in tokens:
        if t == vocabulary[0]:
            counts[0] = counts[0] + 1
        elif t == vocabulary[1]:
            counts[1] = counts[1] + 1
        elif t == vocabulary[2]:
            counts[2] = counts[2] + 1
        else:
            counts[3] = counts[3] + 1
    return np.array(counts)

In [29]:
## Count the chunk occurrences
## unigram, bigram, and trigram may be redundant here.  They all look the same.
sentiment_feats['uni_chunk_counts'] = sentiment_feats['uni_chunk_labels'].apply(countChunks)
sentiment_feats['bi_chunk_counts'] = sentiment_feats['bi_chunk_labels'].apply(countChunks)
sentiment_feats['tri_chunk_counts'] = sentiment_feats['tri_chunk_labels'].apply(countChunks)
sentiment_feats[['uni_chunk_counts', 'bi_chunk_counts', 'tri_chunk_counts']].head()

,uni_chunk_counts,bi_chunk_counts,tri_chunk_counts
0,"[11, 3, 3, 0]","[11, 3, 3, 0]","[11, 3, 3, 0]"
1,"[6, 1, 3, 0]","[6, 1, 3, 0]","[6, 1, 3, 0]"
2,"[4, 2, 0, 0]","[4, 2, 0, 0]","[4, 2, 0, 0]"
3,"[4, 1, 1, 0]","[4, 1, 1, 0]","[4, 1, 1, 0]"
4,"[2, 3, 0, 0]","[2, 3, 0, 0]","[2, 3, 0, 0]"


In [30]:
## Combine all the features
combined_feats = ['pos_unigrams_1hot_tr', 'pos_bigrams_1hot_tr', 'pos_trigrams_1hot_tr', 'pos_unigrams_count_tr','pos_bigrams_count_tr', 'pos_trigrams_count_tr', 'pos_unigrams_tfidf', 'pos_bigrams_tfidf', 'pos_trigrams_tfidf', 'uni_chunk_counts', 'bi_chunk_counts', 'tri_chunk_counts']
sentiment_feats['combined'] = sentiment_feats[combined_feats].apply(np.concatenate, axis=1)
for x in combined_feats:
    print("dim " + x + ":\t", sentiment_feats[x].iloc[0].shape)
print("dim combined:\t\t\t", sentiment_feats["combined"].iloc[0].shape)

dim pos_unigrams_1hot_tr:	 (25,)
dim pos_bigrams_1hot_tr:	 (15,)
dim pos_trigrams_1hot_tr:	 (14,)
dim pos_unigrams_count_tr:	 (25,)
dim pos_bigrams_count_tr:	 (15,)
dim pos_trigrams_count_tr:	 (14,)
dim pos_unigrams_tfidf:	 (25,)
dim pos_bigrams_tfidf:	 (15,)
dim pos_trigrams_tfidf:	 (14,)
dim uni_chunk_counts:	 (4,)
dim bi_chunk_counts:	 (4,)
dim tri_chunk_counts:	 (4,)
dim combined:			 (174,)


In [31]:
from sklearn.model_selection import StratifiedShuffleSplit

split_data=StratifiedShuffleSplit(n_splits=5, test_size=0.2)
for tr_indx, ts_indx in split_data.split(sentiment_feats, sentiment_feats["label"]):
    train_data=sentiment_feats.reindex(tr_indx)
    test_data=sentiment_feats.reindex(ts_indx)

# Set the train and test data
X_train = train_data['combined'].tolist()
y_train = train_data['label'].tolist()
X_test = test_data['combined'].tolist()
y_test = test_data['label'].tolist()

# Train the classifiers and print results
def print_results(name, classifier, X_train, y_train, X_test, y_test):
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    print("accuracy metrics for " + name + " classifier:\n",metrics.classification_report(y_test, predicted, target_names = ['1','0','-1']))
    print("======================================================================================")

print_results("naive bayes", MultinomialNB(), X_train, y_train, X_test, y_test)
print_results("logistic regression", LogisticRegression(), X_train, y_train, X_test, y_test)    
print_results("linear SVM", LinearSVC(), X_train, y_train, X_test, y_test)
print_results("random forest", RandomForestClassifier(), X_train, y_train, X_test, y_test)

accuracy metrics for naive bayes classifier:
               precision    recall  f1-score   support

           1       0.22      0.33      0.26       316
           0       0.15      0.02      0.04       241
          -1       0.92      0.93      0.92      5110

    accuracy                           0.86      5667
   macro avg       0.43      0.43      0.41      5667
weighted avg       0.85      0.86      0.85      5667



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy metrics for logistic regression classifier:
               precision    recall  f1-score   support

           1       0.47      0.06      0.11       316
           0       0.00      0.00      0.00       241
          -1       0.91      1.00      0.95      5110

    accuracy                           0.90      5667
   macro avg       0.46      0.35      0.35      5667
weighted avg       0.84      0.90      0.86      5667



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy metrics for linear SVM classifier:
               precision    recall  f1-score   support

           1       0.29      0.01      0.02       316
           0       0.00      0.00      0.00       241
          -1       0.90      1.00      0.95      5110

    accuracy                           0.90      5667
   macro avg       0.40      0.34      0.32      5667
weighted avg       0.83      0.90      0.86      5667

accuracy metrics for random forest classifier:
               precision    recall  f1-score   support

           1       0.67      0.38      0.48       316
           0       0.77      0.39      0.51       241
          -1       0.94      0.99      0.96      5110

    accuracy                           0.93      5667
   macro avg       0.79      0.58      0.65      5667
weighted avg       0.92      0.93      0.92      5667



## Sentiment Analysis with TextBlob

In [ ]:
from textblob import TextBlob
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer
from textblob.np_extractors import ConllExtractor

import nltk
nltk.download('movie_reviews')
nltk.download('conll2000')

In [ ]:
def extractTextBlobFeatures(corpus):
    extractor = ConllExtractor()
    text_blob_features = np.zeros((len(corpus),4))
    blob_sentiment_analyzer = Blobber(analyzer=NaiveBayesAnalyzer())
    for i,each_text in enumerate(corpus):
        text_blob_features[i,0]=blob_sentiment_analyzer(each_text).sentiment[1]
        text_blob_features[i,1]=blob_sentiment_analyzer(each_text).sentiment[2]
        text_blob_features[i,2]= TextBlob(each_text).subjectivity
        noun_phrase_extractor = TextBlob(each_text, np_extractor=extractor)
        text_blob_features[i,3]= len(noun_phrase_extractor.noun_phrases)
    return text_blob_features

In [ ]:
X_train=train_data['reviews.text']
y_train=train_data["sentiment_label"]
X_test=test_data["reviews.text"]
y_test=test_data["sentiment_label"]

In [ ]:
Xtr_tb_features = extractTextBlobFeatures(X_train)
Xte_tb_features = extractTextBlobFeatures(X_test)
ytr_tb_features = extractTextBlobFeatures(y_train)
yte_tb_features = extractTextBlobFeatures(y_test)

In [ ]:
Xtr_tb_features

In [ ]:
# Xtr= np.array(Xtr_tb_features.ravel(),)
# ytr=np.array(ytr_tb_features.ravel(),)

# Xte = np.array(Xte_tb_features.ravel(),)
# yte = np.array(yte_tb_features.ravel(),)

In [ ]:
# ## use multinomial NB classifier
# clf_multiNB_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_nominalNB", MultinomialNB())])
# clf_multiNB_pipe.fit(Xtr, ytr)
# predicted_nb = clf_multiNB_pipe.predict(Xte)
# print("accuracy metrics for training naive bayes classifier:\n",metrics.classification_report(yte, predicted_nb, target_names = ['1','0','-1']))
# print("======================================================================================")

In [ ]:
Xte_tb_features

In [ ]:
ytr_tb_features